In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 13.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661656 sha256=ff5e50c30d06fbda322208bfecad7a30877ffa593ab2cafdc8dd35305f6d2064
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [2]:
import os

# 수정할 파일 경로
file_path = "/usr/local/lib/python3.10/dist-packages/mxnet/numpy/utils.py"

# 수정할 줄과 변경할 내용
target_line = "bool = onp.bool"  # 기존 줄
replacement_line = "bool = bool"  # 변경할 내용

# 파일 수정
if os.path.exists(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    # 줄 수정
    with open(file_path, "w") as file:
        for line in lines:
            if target_line in line:
                file.write(line.replace(target_line, replacement_line))
            else:
                file.write(line)
    print(f"{file_path} 파일의 '{target_line}'을(를) '{replacement_line}'으로 수정했습니다.")
else:
    print(f"파일 {file_path}을(를) 찾을 수 없습니다.")


/usr/local/lib/python3.10/dist-packages/mxnet/numpy/utils.py 파일의 'bool = onp.bool'을(를) 'bool = bool'으로 수정했습니다.


In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-2oof4wwf/kobert-tokenizer_509b2f67944d46a98b3845d48f5fac6f
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-2oof4wwf/kobert-tokenizer_509b2f67944d46a98b3845d48f5fac6f
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 5c46b1c68e4755b54879431bd302db621f4d2f47
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=ba238045b9b46c30305f75b053f15dc12ce753a613185780c59bdf185cae2db2
  Stored in directory: /tmp/pip-ephem-wheel-cache-162nxdbl/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:

from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
device = torch.device("cuda:0")

In [7]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [8]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2024-12-09 04:52:45--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/5d9cfl614kbxy44g293bz/ratings_train.txt?rlkey=ve890xjd54zg9giz317qjycv6&dl=1 [following]
--2024-12-09 04:52:46--  https://www.dropbox.com/scl/fi/5d9cfl614kbxy44g293bz/ratings_train.txt?rlkey=ve890xjd54zg9giz317qjycv6&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucee145e5ef5fb58db156ce2f249.dl.dropboxusercontent.com/cd/0/inline/Cf4rqsMNq3UjZoXJiHB2-373Sthf9JMQDmW-wRmoqR6AbFWvorEu-HY747NpM4T1WHK6N26h_EO1MSqS6eElVhm8elDu2wA52nEF0DgNsvztCXHR0zopYpysmvALhlbJLik/file?dl=1# [following]
--2024-12-09 04:52:46--  https://ucee145e5ef5fb58db156ce2f249.dl.dropboxusercontent.com

In [9]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [10]:
# ★
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [11]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [12]:
# ★
tok = tokenizer.tokenize

data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [13]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [14]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [15]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [16]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [17]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [19]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-19-e6a38b13095b>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7370320558547974 train acc 0.515625
epoch 1 batch id 201 loss 0.5052235126495361 train acc 0.5668532338308457
epoch 1 batch id 401 loss 0.36039888858795166 train acc 0.6797459476309227
epoch 1 batch id 601 loss 0.4457674026489258 train acc 0.7310212146422629
epoch 1 batch id 801 loss 0.39062997698783875 train acc 0.7611189138576779
epoch 1 batch id 1001 loss 0.3276486396789551 train acc 0.7791115134865135
epoch 1 batch id 1201 loss 0.3459911346435547 train acc 0.7922174229808493
epoch 1 batch id 1401 loss 0.35161060094833374 train acc 0.8013360992148465
epoch 1 batch id 1601 loss 0.3144916296005249 train acc 0.8095233447845097
epoch 1 batch id 1801 loss 0.2532314360141754 train acc 0.8163780538589672
epoch 1 batch id 2001 loss 0.3200337886810303 train acc 0.8220811469265368
epoch 1 batch id 2201 loss 0.28716737031936646 train acc 0.8271524307133121
epoch 1 train acc 0.8305802936575654


<ipython-input-19-e6a38b13095b>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/782 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8980578644501279


  0%|          | 0/2344 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.26557493209838867 train acc 0.90625
epoch 5 batch id 201 loss 0.0371994823217392 train acc 0.9729477611940298
epoch 5 batch id 401 loss 0.0482078492641449 train acc 0.9743609725685786
epoch 5 batch id 601 loss 0.15360458195209503 train acc 0.9746256239600666
epoch 5 batch id 801 loss 0.0520290769636631 train acc 0.9755578963795256
epoch 5 batch id 1001 loss 0.10875911265611649 train acc 0.9765703046953047
epoch 5 batch id 1201 loss 0.010713201947510242 train acc 0.9771154246461282
epoch 5 batch id 1401 loss 0.09693946689367294 train acc 0.9773599214846538
epoch 5 batch id 1601 loss 0.051180094480514526 train acc 0.9777678013741412
epoch 5 batch id 1801 loss 0.0790325254201889 train acc 0.9783887423653526
epoch 5 batch id 2001 loss 0.07787538319826126 train acc 0.978713768115942
epoch 5 batch id 2201 loss 0.026921626180410385 train acc 0.9788590413448433
epoch 5 train acc 0.9790622333617748


  0%|          | 0/782 [00:00<?, ?it/s]

epoch 5 test acc 0.8976782289002557


In [20]:
model.bert.save_pretrained("model")
tokenizer.save_pretrained("model")

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/spiece.model',
 'model/added_tokens.json')

In [26]:
!zip -r model.zip model


  adding: model/ (stored 0%)
  adding: model/spiece.model (deflated 46%)
  adding: model/model.safetensors (deflated 7%)
  adding: model/special_tokens_map.json (deflated 48%)
  adding: model/config.json (deflated 47%)
  adding: model/tokenizer_config.json (deflated 74%)


In [25]:
!ls model/


config.json  model.safetensors	special_tokens_map.json  spiece.model  tokenizer_config.json


In [27]:
def predict_sentiment(sentence):
    # 토큰화 수행
    encoded = tokenizer(
        sentence,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    )

    # 모델에 필요한 입력 데이터 생성
    token_ids = encoded["input_ids"].to(device)
    segment_ids = encoded["token_type_ids"].to(device)
    valid_length = (token_ids != tokenizer.pad_token_id).sum(dim=1)  # 패딩 제외한 유효 길이 계산

    # 예측 수행
    model.eval()
    with torch.no_grad():
        outputs = model(token_ids, valid_length, segment_ids)
        probabilities = torch.softmax(outputs, dim=1)
        label = torch.argmax(probabilities).item()

    return "긍정" if label == 1 else "부정"


In [32]:
# 8. 테스트
test_sentence = "서승현 개새끼!!!!"
result = predict_sentiment(test_sentence)
print(f"문장: {test_sentence}\n결과: {result}")


문장: 서승현 개새끼!!!!
결과: 긍정
